In [2]:
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
def obtener_datos_tabla(url):
    # Realizar la solicitud HTTP y obtener el contenido de la página
    res = requests.get(url)
    
    # Creo mi sopa
    sopa = BeautifulSoup(res.content, "html.parser")
    
    # Buscar el bloque de datos en el HTML
    bloque_total = sopa.findAll("tbody")
    
    lista_de_tuplas = []
    # Iterar sobre los bloques de datos
    for i in bloque_total:
        filas = i.findAll("tr")
        
        for fila in filas:
            # Extraer todas las celdas de la fila
            lista_td = fila.findAll("td")
            # Crear una lista intermedia para procesar los datos
            lista_intermedia = [elemento.text for elemento in lista_td]
            # Añadimos a mi tupla, nutriendonos de la url, el supermercado, la categoria y el producto
            lista_intermedia.extend(url.split("/")[3:6])
            # Convertir el segundo elemento a float y reemplazar ',' por '.'
            lista_intermedia[1] = float(lista_intermedia[1].replace(",", "."))
            
            # Agregar los datos a la lista de tuplas
            lista_de_tuplas.append(tuple(lista_intermedia))
    
    return lista_de_tuplas 


In [4]:
def get_urls(url, texto):
    driver = webdriver.Chrome()
    urls_return = []

    driver.get(url)
    driver.maximize_window()
    sleep(5)
    driver.find_element('css selector','#rcc-confirm-button').click()
    sleep(5)
    links = driver.find_elements(By.PARTIAL_LINK_TEXT, texto)

    for i in links:
        urls_return.append(i.get_attribute("href"))

    driver.close()

    return urls_return

In [5]:
listas_url_supermercados = get_urls("https://super.facua.org/", 'Acceder')

In [6]:
listas_categorias_por_supermercados1 = []
for i in listas_url_supermercados:
    listas_categorias_por_supermercados1.append(get_urls(i,'Ver'))

In [7]:
lista_limpita_categorias = []
for k in listas_categorias_por_supermercados1:
    for h in k:
        lista_limpita_categorias.append(h)

In [8]:
listas_productos = []
for producto in lista_limpita_categorias:
    listas_productos.append(get_urls(producto, 'Histórico'))

In [9]:
lista_productos_limpiados = []
for p in listas_productos:
    for q in p:
        lista_productos_limpiados.append(q)